In [14]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
from glob import glob

## Loading and merging data

In [15]:
# df = pd.DataFrame()
# for file in glob("shifts_123_valid/*.csv"):
#     shift_df = pd.read_csv(file)
#     col = ["timeleft", "shift","tostop", "train_test","stoplon", "stoplat", "Lon", "Lat", 'Timestamp', "speed_smooth_10", "timepast", "delay", "deltalon", "deltalat","tostop_estimate","time_avg"]
#     shift_df = shift_df[col]
#     shift_df['Timestamp'] = pd.to_datetime(shift_df['Timestamp'])
#     shift_df["hour"] = shift_df["Timestamp"].apply(lambda x: x.hour)
#     shift_df["weekday"] = shift_df["Timestamp"].apply(lambda x: x.weekday())
#     df = pd.concat([df, shift_df],ignore_index=True)

    
# with open('data/shifts_new_123.pickle', 'wb') as handle:
#     pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [45]:
with open('data/shifts_new_123.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [17]:
missing_columns = df.columns[df.isnull().any()].tolist()
df = df.dropna(subset=missing_columns,how='any')
df = df.dropna(how='any') 
df["tostop_estimate"] = df["tostop_estimate"] - df["time_avg"]

## Generating faeture matrix for CNN model


In [23]:

shifts_list = df["shift"].unique().tolist()
stops_list = df["tostop"].unique().tolist()
start = time.time()
count = 0
for shift in shifts_list[:]:
    sub_df = df[df["shift"]==shift]

    df_out = pd.DataFrame()
    for stop in stops_list:
        sub_sub_df = sub_df[sub_df["tostop"] == stop]
        try:
            test = sub_sub_df["train_test"].tolist()[0]
        except:
            continue
        sub_sub_df["deltalon"] = sub_sub_df["stoplon"] - sub_sub_df["Lon"]
        sub_sub_df["deltalat"] = sub_sub_df["stoplat"] - sub_sub_df["Lat"]
        sub_sub_df = sub_sub_df.reset_index().drop("index", axis = 1)

        cache_df = pd.DataFrame()
        for index, row in sub_sub_df.iterrows():
            if index % 10 == 0:
                cur_row = row[["Lon", "Lat", "hour", "weekday", "speed_smooth_10", "timepast", "delay", "deltalon", "deltalat","tostop_estimate"]]
                cur_row = pd.DataFrame(np.array(cur_row).reshape(1,10))
                
                cache_df = pd.concat([cache_df, cur_row] , ignore_index=True)               
                if cache_df.shape[0] == 10:
                    df_out = pd.concat([df_out, pd.DataFrame(np.append(cache_df.as_matrix().reshape((1,100)), row["timeleft"]).reshape((1,101)))] , ignore_index=True)
                    cache_df = cache_df.loc[1:,]

    if test:
        df_out.to_csv("cnndata_123/"+str(shift)+"_test.csv", index=False, header = False)
    else:
        df_out.to_csv("cnndata_123/"+str(shift)+"_train.csv", index=False, header = False)


/Users/yuanmh/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yuanmh/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
